# Data Import and Preview

In [1]:
import pandas as pd
df=pd.read_csv('data.csv', sep=',')
print(df.shape)

(1027848, 170)


In [2]:
df.head()

,Cell_ID,Animal_ID,Animal_sex,Behavior,Bregma,Centroid_X,Centroid_Y,Cell_class,Neuron_cluster_ID,Ace2,...,Penk,Scg2,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf
0,6749ccb4-2ed1-4029-968f-820a287f43c8,1,Female,Naive,0.26,-3211.562145,2608.541476,Astrocyte,NaN,0.0,...,0.133016,0.000000,0.865263,0.002977,0.054826,0.008934,0.0,0.0,0.0,0.000000
1,6cac74bd-4ea7-4701-8701-42563cc65eb8,1,Female,Naive,0.26,-3207.923151,2621.795437,Inhibitory,I-5,0.0,...,0.000000,0.000000,0.277939,0.868702,0.580957,0.010079,0.0,0.0,0.0,0.000000
2,9f29bd57-16a5-4b26-b9f5-37598809da9e,1,Female,Naive,0.26,-3209.578004,2633.153494,Inhibitory,I-6,0.0,...,0.213939,0.000000,0.377907,0.049332,0.084898,0.008951,0.0,0.0,0.0,0.000000
3,d7eb4e0b-276e-47e3-a55c-0b033180a2fe,1,Female,Naive,0.26,-3203.853515,2756.045983,Inhibitory,I-5,0.0,...,0.050882,0.089038,0.000000,0.000000,0.001530,0.031364,0.0,0.0,0.0,0.001138
4,54434f3a-eba9-4aec-af35-c9d317ffa1d5,1,Female,Naive,0.26,-3202.682705,2608.803635,Inhibitory,I-9,0.0,...,1.250661,0.159618,0.211159,0.000000,0.087730,0.000000,0.0,0.0,0.0,0.029419


# Data Processing

Bad values processing

In [3]:
train_size = 27848 #subject to change, I set it to 27848 to run the classifer faster for demo purpose only
train_data = df[0:train_size]
train_data = train_data.fillna(0)

Create a Training Set

In [4]:
y = train_data["Cell_class"]
X = train_data.drop(['Cell_ID','Animal_ID', 'Animal_sex', 'Behavior','Bregma','Centroid_X','Centroid_Y','Cell_class','Neuron_cluster_ID'], axis = 1)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train.head()


,Ace2,Adora2a,Aldh1l1,Amigo2,Ano3,Aqp4,Ar,Arhgap36,Avpr1a,Avpr2,...,Penk,Scg2,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf
9914,3.877194,0.000000,0.000000,3.877194,0.000000,0.000000,0.000000,3.877194,0.000000,0.000000,...,0.000000,0.029227,0.000000,0.003443,0.000000,0.013528,0.000000,0.012759,0.000000,0.004307
13826,0.837934,0.837934,1.675839,14.244037,0.000000,0.000000,0.837934,4.189673,0.000000,1.675839,...,0.258305,0.251567,0.633729,0.000000,0.047681,0.000000,0.000000,0.000000,0.001424,0.033644
13431,0.000000,2.464097,7.392474,0.000000,0.000000,9.856754,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.322848,0.021557,0.050253,0.028720,0.043006,0.000000,0.019580,0.004159
6173,0.000000,0.000000,0.000000,0.943838,1.887686,0.000000,2.831599,2.831599,0.943838,0.000000,...,0.000000,0.000000,0.000000,0.101489,0.000000,0.004942,0.001107,0.000000,0.004772,0.043290
17696,0.000000,0.000000,1.492470,0.000000,0.746224,0.746224,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.027983,0.000000,0.000000,0.000000,0.035815,0.022926,0.028608


# Clustering

Necessary Imports

In [6]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.image import grid_to_graph
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn import metrics

Create and fit the clustering models

In [7]:
linkMethods = ['complete','average','single']
n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]

models = []
si = []
ar = []
fm = []
ch = []

for cluster in n_clusters:
    tempModels = []
    tempSi = []
    #tempAr = []
    #tempFm = []
    tempCh = []
    
    for method in linkMethods:
        model = AgglomerativeClustering(n_clusters=cluster, linkage=method)
        fitted_model = model.fit(X_train)
        tempModels = tempModels + [fitted_model]
        tempSi = tempSi + [metrics.silhouette_score(X_train, model.labels_)]
        #tempAr = tempAr + [metrics.adjusted_rand_score(X_train, model.labels_)]
        #tempFm = tempFm + [metrics.fowlkes_mallows_score(X_train, model.labels_)]
        tempCh = tempCh + [metrics.calinski_harabasz_score(X_train, model.labels_)]
        
    models = models + [tempModels]
    si = si + [tempSi]
    #ar = ar + [tempAr]
    #fm = fm + [tempFm]
    ch = ch + [tempCh]


In [8]:
numClusters = len(n_clusters)
numMethods = len(linkMethods)

(si_Cluster, si_Method, si_Val, si_cIdx, si_mIdx) = (2,'complete', si[0][0], 0, 0)
(ch_Cluster, ch_Method, ch_Val, ch_cIdx, ch_mIdx) = (2,'complete', ch[0][0], 0, 0)

c = 0
for cluster in n_clusters:
    
    m = 0
    for method in linkMethods:
        if si[c][m] > si_Val:
            (si_Cluster, si_Method, si_Val, si_cIdx, si_mIdx) = (cluster, method, si[c][m], c, m)
        
        if ch[c][m] > ch_Val:
            (ch_Cluster, ch_Method, ch_Val, ch_cIdx, ch_mIdx) = (cluster, method, ch[c][m], c, m)
            
        m = m+1
    
    c = c+1
    
print((si_Cluster, si_Method, si_Val, si_cIdx, si_mIdx))
print((ch_Cluster, ch_Method, ch_Val, ch_cIdx, ch_mIdx))

(2, 'complete', 0.8293823755973952, 0, 0)
(4, 'complete', 654.0087839218514, 2, 0)


Function which plots the dendogram

In [9]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [12]:
Z = linkage(X_train, ch_Method)
dendrogram(Z,  truncate_mode='level',p=ch_Cluster)

#plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
#plot_dendrogram(models[ch_cIdx][ch_mIdx], truncate_mode='level')
#plt.xlabel("Number of points in node (or index of point if no parenthesis).")

{'icoord': [[25.0, 25.0, 35.0, 35.0],
  [45.0, 45.0, 55.0, 55.0],
  [30.0, 30.0, 50.0, 50.0],
  [15.0, 15.0, 40.0, 40.0],
  [65.0, 65.0, 75.0, 75.0],
  [85.0, 85.0, 95.0, 95.0],
  [70.0, 70.0, 90.0, 90.0],
  [105.0, 105.0, 115.0, 115.0],
  [125.0, 125.0, 135.0, 135.0],
  [110.0, 110.0, 130.0, 130.0],
  [80.0, 80.0, 120.0, 120.0],
  [27.5, 27.5, 100.0, 100.0],
  [5.0, 5.0, 63.75, 63.75]],
 'dcoord': [[0.0, 80.19991751621416, 80.19991751621416, 0.0],
  [0.0, 108.61743663144297, 108.61743663144297, 0.0],
  [80.19991751621416,
   155.01143895089265,
   155.01143895089265,
   108.61743663144297],
  [0.0, 244.17470672434175, 244.17470672434175, 155.01143895089265],
  [0.0, 188.47715827630185, 188.47715827630185, 0.0],
  [0.0, 227.93937603306858, 227.93937603306858, 0.0],
  [188.47715827630185,
   238.26787489205785,
   238.26787489205785,
   227.93937603306858],
  [0.0, 235.47838570193065, 235.47838570193065, 0.0],
  [0.0, 321.26310026248956, 321.26310026248956, 0.0],
  [235.47838570193065,
